### 디버깅 또는 실행 속도 모드 설정
0 = debug(가장 느림), 1 = normal run, 2 = run fast, 3 = without camera view(가장 빠름)

In [1]:
runMode = 0

### OLED 디스플레이 (Adafruit OLED, SSD1306, 128 x 32)

In [2]:
import Adafruit_SSD1306
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

from jetbot.utils.utils import get_ip_address
import subprocess
import time

##### OLED-SSD1306 초기화

In [3]:
# 하드웨어 I2C 방식의 128x32 display
# gpio는 OLED 디바이스 인식문제 해결을 위해 1로 설정
disp = Adafruit_SSD1306.SSD1306_128_32(rst=None, i2c_bus=0, gpio=1)

# 디스플레이 초기화
disp.begin()

# 화면 클리어
disp.clear()
disp.display()

# 디스플레이 이미지 생성, 흑백(1-bit) 이미지 사용
disp_image = Image.new('1', (disp.width, disp.height))

# 이미지에 그릴 드로잉 객체 얻기
screen = ImageDraw.Draw(disp_image)

# 화면 전체를 검은색으로 채워서 지우기
screen.rectangle((0, 0, disp.width, disp.height), outline=0, fill=0)

# 화면 패딩영역 위치 조정(글씨가 짤리지 않도록)
screen_padding = 2
screen_top = screen_padding
screen_bottom = disp.height - screen_padding
screen_x = 0

# 기본 서체 사용
font = ImageFont.load_default()

##### OLED 화면에 기본 정보 디스플레이

In [4]:
### OLED 화면에 시스템 정보 4줄을 보여주는 함수

def ShowInfo(strInfo):
    # 화면 전체를 검은색으로 채워서 지우기
    screen.rectangle((0, 0, disp.width, disp.height), outline=0, fill=0)
    
    # 화면 맨 위의 첫번째 줄에는 매개변수로 넘겨받은 문자열을 디스플레이
    screen.text((screen_x, screen_top+0), strInfo, font=font, fill=255)
    
    # 두번째 줄에는 젯봇의 IP Address를 디스플레이
    screen.text((screen_x, screen_top+8), "IP:" + str(get_ip_address('wlan0')), font=font, fill=255)
    
    # 세번째 줄에는 메모리 사용 상태를 디스플레이
    cmd = "free -m | awk 'NR==2{printf \"Mem:%s/%sM %.2f%%\", $3, $2, $3*100/$2 }'"
    MemUsage = subprocess.check_output(cmd, shell = True)
    screen.text((screen_x, screen_top+16), str(MemUsage.decode('utf-8')), font=font, fill=255)
    
    # 네번째 줄에는 디스크 사용 상태를 디스플레이
    cmd = "df -h | awk '$NF==\"/\"{printf \"Disk:%d/%dGB %s\", $3, $2, $5}'"
    Disk = subprocess.check_output(cmd, shell = True)
    screen.text((screen_x, screen_top+25), str(Disk.decode('utf-8')), font=font, fill=255)
    
    # 설정한 내용을 OLED 화면으로 출력
    disp.image(disp_image)
    disp.display()
    
# 맨 위의 1번째 줄에 텍스트를 디스플레이 하는 함수
def ShowTopInfo(strInfo):
    screen.rectangle((0, 0, disp.width, 7), outline=0, fill=0)
    screen.text((screen_x, screen_top), strInfo, font=font, fill=255)
    disp.image(disp_image)
    disp.display()
    
if runMode < 2:
    ShowInfo("Init...")

In [6]:
ShowInfo('hello')

### RGB Strip LED 생성
젯봇 좌우의 무한궤도 캐터필러 안쪽에 장착되어 있는 5x2 LED 제어

In [7]:
from RGB_Lib import Programing_RGB
RGB = Programing_RGB()

#### 개별 1개 LED 또는 전체 LED 색상 설정

##### LED는 인덱스가 0번부터 9번까지 전부 10개
##### 오른쪽: 뒤쪽[0] > [1] > [2] > [3] > 앞쪽[4] / 왼쪽: 뒤쪽[5] > [6] > [7] > [8] > 앞쪽[9]
##### 뒷 페이지의 순차흐름, 무지개, 숨쉬기 LED Show는 생략

In [7]:
# if runMode < 2:
#     RGB.Set_An_RGB(0, 0xFF, 0x00, 0x00) # 0번째 LED (우측 뒤쪽) 빨강
#     RGB.Set_An_RGB(4, 0x00, 0xFF, 0x00) # 0번째 LED (우측 앞쪽) 초록
#     RGB.Set_An_RGB(5, 0x00, 0x00, 0xFF) # 0번째 LED (좌측 뒤쪽) 파랑
#     RGB.Set_An_RGB(9, 0xFF, 0xFF, 0xFF) # 0번째 LED (좌측 앞쪽) 흰색

#     RGB.Set_All_RGB(0xFF, 0x00, 0x00) # 전체 LED 색상을 설정
#     time.sleep(0.5)
#     RGB.Set_All_RGB(0x00, 0xFF, 0x00) # 전체 LED 색상을 설정
#     time.sleep(0.5)
#     RGB.Set_All_RGB(0x00, 0x00, 0xFF) # 전체 LED 색상을 설정
#     time.sleep(0.5)
    
# RGB.OFF_ALL_RGB() # 전체 LED 끄기
# time.sleep(1)


### I/O 제어: 부저, 버튼, 단색 LED

In [60]:
import RPi.GPIO as GPIO

BEEP_pin = 6 # Buzzer 핀 번호

GREEN_pin = 24 # BOARD pin 24, GREEN (LED2)
BLUE_pin = 23 # BOARD pin 23, Blue (LED3)

KEY1_pin = 8 # BOARD pin 8, K1 버튼
KEY2_pin = 7 # BOARD pin 7, K2 버튼

SW_UP_pin = 17 # Camera Lifter 상단 Limit Switch
SW_DOWN_pin = 18 # Camera Lifter 하단 limit Switch

GPIO.setmode(GPIO.BCM) # 보드 핀 번호 방식

GPIO.setup(BEEP_pin, GPIO.OUT, initial=GPIO.LOW) # 초기설정 = 끄기

GPIO.setup(GREEN_pin, GPIO.OUT) # 초록 LED Pin set as output
GPIO.setup(BLUE_pin, GPIO.OUT) # 파랑 LED Pin set as output

GPIO.setup(KEY1_pin, GPIO.OUT) # KEY1 버튼 Pin set as output
GPIO.setup(KEY2_pin, GPIO.OUT) # KEY2 버튼 Pin set as output

In [61]:
### 지정 시간 동안 부저음 울리기
def Beep(beepTime):
    GPIO.output(BEEP_pin, GPIO.HIGH) # 부저 켜기
    time.sleep(beepTime) # 소리 내는 시간
    GPIO.output(BEEP_pin, GPIO.LOW) # 부저 끄기
    
### 보드상의 초록 LED on/off
def OnOffGreen(bOn):
    if bOn:
        GPIO.output(GREEN_pin, GPIO.LOW) # GREEN LED 켜기
    else:
        GPIO.output(GREEN_pin, GPIO.HIGH) # GREEN LED 켜기
                    
### 보드상의 파랑 LED on/off
def OnOffBlue(bOn):
    if bOn:
        GPIO.output(BLUE_pin, GPIO.LOW) # BLUE LED 켜기
    else:
        GPIO.output(BLUE_pin, GPIO.HIGH) # BLUE LED 켜기
                    


In [62]:
OnOffGreen(1)
OnOffBlue(1)
Beep(1)

In [11]:
# 기둥 위아래로 moving

from jetbot import Robot
robot = Robot()

OnOffGreen(1)
OnOffBlue(1)

value = GPIO.input(KEY1_pin) # GPIO.LOW or GPIO.HIGH
value = GPIO.input(KEY2_pin) # GPIO.LOW or GPIO.HIGH

GPIO.setwarnings(False)
GPIO.setup(SW_UP_pin, GPIO.IN)
GPIO.setup(SW_DOWN_pin, GPIO.IN)

# if GPIO.input(SW_UP_pin):
#     robot.down(1)
# else:
#     robot.vertical_motors_stop()
    
if GPIO.input(SW_DOWN_pin):
    robot.up(1)
else:
    robot.vertical_motors_stop()
    
# GPIO.cleanup()

### 모터/서보 성능 및 위치 기본 조정값 선언

In [12]:
L_F_MPR = 0.93 # 좌측 전진 모터 성능 비율 (Left_forward_Motor_Performance_Ratio)
R_F_MPR = 0.93 # 우측 전진 모터 성능 비율 (Right_forward_Motor_Performance_Ratio)

L_B_MPR = 0.98 # 좌측 후진 모터 성능 비율 (Left_Backward_Motor_Performance_Ratio)
R_B_MPR = 0.98 # 우측 후진 모터 성능 비율 (Right_Backward_Motor_Performance_Ratio)

PAN_ADJ = 0 # 카메라 팬 서보 위치 조정값(Camera_Pan_Servo_Adjust)
TILT_ADJ = 0 # 카메라 틸트 서보 위치 조정값(Camera_Tilt_Servo_Adjust)

##### 모터/서보 성능 위치 및 조정정보 설정파일 읽기
생략

### PanTilt 서보 인스턴스 생성

In [13]:

from servoserial import ServoSerial
pantilt = ServoSerial() # 팬틸트 서보 컨트롤

serial Open!


##### PAN, TILT 값 정의

In [14]:
MID_TILT = 3600
# TILT 범위 3000~4095
MID_PAN = 4095 // 2 + 100

DEF_TILT = 4000

### 초기 카메라 팬/틸트 위치 설정


In [15]:
# pantilt.Servo_serial_double_control(1, MID_PAN, 2, MID_TILT-600) # 장애물회피 주행용 팬틸트 위치
# time.sleep(1)

# pantilt.Servo_serial_double_control(1, MID_PAN, 2, MID_TILT) # 중앙정면 팬틸트 위치
# time.sleep(1)

pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틸트 위치
time.sleep(1)

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'


### Robot 인스턴스는 위에서 생성했음

##### Camera Lifter 테스트
젯봇 전면의 카메라 높낮이를 조정하는 리프터 상하 이동 제어

In [16]:
# robot.up(1.0)
# time.sleep(3)
# robot.vertical_motors_stop()

# robot.down(1.0)
# time.sleep(3)
# robot.vertical_motors_stop()

##### 기본 주행 속력과 주행 및 정지 함수 정의

In [16]:
Speed = 0.7 ## 최대 속력의 70% 빠르기를 기본 속력으로 설정

### 좌우 모터의 속력을 따로 지정하여 주행하기

def Move2(speedLeft, speedRight):
    if speedLeft > 0:
        robot.left_motor.value = speedLeft * L_F_MPR # 전진 시 왼쪽 모터의 전진 성능 비율 적용
    else:
        robot.left_motor.value = speedLeft * L_B_MPR # 후진 시 왼쪽 모터의 후진 성능 비율 적용
    
    if speedRight > 0:
        robot.right_motor.value = speedRight * R_F_MPR # 전진 시 오른쪽 모터의 전진 성능 비율 적용
    else:
        robot.right_motor.value = speedRight * R_B_MPR # 후진 시 오른쪽 모터의 후진 성능 비율 적용
    
### 좌우 모터의 속력을 동일하게 지정하여 주행하기

def Move(speed):
    Move2(speed, speed)
    
### 이동 정지
def Stop():
    robot.left_motor.value = 0
    robot.right_motor.value = 0
    


##### MODE_PERSON 동작: 사람이 감지되면 반가워서 인사하는 동작
사람이 마지막에 감지된 시각으로부터 특정 시간보다 더 지나서 다시 사람이 감지되면 동작됨

In [17]:
ActTimePeriodPerson = 5 # 30초 이상 사람이 감지되지 않다가 다시 감지되면 동작
LastTimeDetectedPerson = 0 # 가장 최근에 사람이 감지된 시각

def ActModePerson(repeat=3):
    speed = 3
    delay = 0.02
    
    RGB.Set_BreathSColor_RGB(6) # Color: 0(red), 1(green), 2(blue), 3(yellow), 4(magenta), 5(cyan), 6(white)
    RGB.Set_BreathSSpeed_RGB(3) # speed: 1(slow), 2(normal), 3(fast)
    RGB.Set_BreathSLight_RGB()
    
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틸트 위치
    Beep(0.05)
    
    for i in range(0, repeat):
        pantilt.Servo_serial_control(1, MID_PAN+200) # Left
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+400)
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+200) 
        Stop()
        time.sleep(delay)
        
        pantilt.Servo_serial_control(1, MID_PAN-200) # Right
        Move2(speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-400)
        Move2(speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-200) 
        Stop()
        time.sleep(delay)
        
        pantilt.Servo_serial_control(1, MID_PAN)
        Stop()
        time.sleep(delay)
        
    for i in range(0, 2):
        pantilt.Servo_serial_control(2, DEF_TILT+100)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT+200)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT+100)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-200)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(delay)
    
    Beep(0.05)
    time.sleep(0.1)
    Beep(0.05)
    
    RGB.OFF_ALL_RGB() # 전체 LED 끄기
    RGB.Set_All_RGB(0x00, 0x00, 0x10) # 기본 색상 어두운 파랑으로

In [18]:
ActModePerson()

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
196
b'\xff\xff\x01\x07\x03*\t\xf3\x00\n\xc4'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
231
b'\xff\xff\x01\x07\x03*\x06\xd3\x00\n\xe7'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
85
b'\xff\xff\x01\x07\x03*\x08c\x00\nU'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
196
b'\xff\xff\x01\x07\x03*\t\xf3\x00\n\xc4'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
231
b'\xff\xff\x01\x07\x03*\x06\xd3\x00\n\xe7'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
85
b'\xff\xff\x01\x07\x03*\x08c\x00\nU'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
196
b'\xff\xff\x01\x07\x03*\t\xf3\x00\n\xc4'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
231
b'\xff\xff\x01\x07\x03*\x06\xd3\x00\n\xe7'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
85
b'\xff\xff\x01\x07\x03*\x08c\x00\nU'
177
b'\

##### ActModeFruit: 과일 먹는 동작

In [19]:
ActTimePeriodFruit = 5 # 30초 이상 컵이 감지되지 않다가 다시 감지되면 동작
LastTimeDetectedFruit = 0 # 가장 최근에 컵이 감지된 시각

def ActModeFruit(repeat = 2):
    speed = 0.4
    delay = 0.05
    
    RGB.Set_BreathSColor_RGB(1) # Color : 0(red), 1(green), 2(blue), 3(yellow), 4(magenta), 5(cyan), 6(white)
    RGB.Set_BreathSSpeed_RGB(3) # speed : 1(slow), 2(normal), 3(fast)
    RGB.Set_BreathSLight_RGB()
    
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT)
    
    for i in range(0, repeat):
        pantilt.Servo_serial_control(2, DEF_TILT-100) # down
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-200) # down
        Move2(speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-300) # down
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-400) # down
        Move2(speed, -speed)
        time.sleep(delay)
        Stop()
        
        # 고개 숙이는 동작
        pantilt.Servo_serial_double_control(1, MID_PAN-50, 2, DEF_TILT-500) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-300) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN+50, 2, DEF_TILT-500) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-300) 
        time.sleep(delay)
        
        pantilt.Servo_serial_double_control(1, MID_PAN-50, 2, DEF_TILT-500)
        Move2(speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-500)
        Move2(-speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN+50, 2, DEF_TILT-500)
        Move2(speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-500)
        Move2(-speed, -speed)
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN-50, 2, DEF_TILT-500)
        Stop()
        time.sleep(delay)
        
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-300) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN+50, 2, DEF_TILT-500) 
        time.sleep(delay)
        pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT-300) 
        time.sleep(delay)
        
        pantilt.Servo_serial_control(2, DEF_TILT-400)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-300)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-200)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-100)
        time.sleep(delay)
        
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(delay)
        
        # 고개 들어올림
        pantilt.Servo_serial_control(1, MID_PAN+100) # left
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+200)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+100)
        time.sleep(delay)
        
        pantilt.Servo_serial_control(1, MID_PAN-100) # right
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-200) 
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-100) 
        time.sleep(delay)
        
        pantilt.Servo_serial_control(1, MID_PAN)
        
        RGB.OFF_ALL_RGB() # 전체 LED 끄기
        RGB.Set_All_RGB(0x00, 0x00, 0x10) # 기본 색상으로 켜기(어두운 파랑)

In [21]:
ActModeFruit()

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
217
b'\xff\xff\x02\x07\x03*\x0e\xd8\x00\n\xd9'
61
b'\xff\xff\x02\x07\x03*\x0et\x00\n='
161
b'\xff\xff\x02\x07\x03*\x0e\x10\x00\n\xa1'
57
b'\xff\xff\xfe\x0e\x83*\x04\x01\x081\x00\n\x02\r\xac\x00\n9'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
213
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x95\x00\n\x02\r\xac\x00\n\xd5'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
57
b'\xff\xff\xfe\x0e\x83*\x04\x01\x081\x00\n\x02\r\xac\x00\n9'
7
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\r\xac\x00\n\x07'
213
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x95\x00\n\x02\r\xac\x00\n\xd5'
7
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\r\xac\x00\n\x07'
57
b'\xff\xff\xfe\x0e\x83*\x04\x01\x081\x00\n\x02\r\xac\x00\n9'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
213
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x95\x00\n\x02\r\xac\x00\n\xd5'
62
b'\xff\xff\xfe\x0e\x83*\x04

#### MODE_CUP 동작: 음료수나 물을 마시는 동작(컵이 감지되었을 때)
컵이 마지막에 감지된 시각으로부터 특정 시간보다 더 지나서 다시 컵이 감지되면 동작됨

In [20]:
ActTimePeriodCup = 5 # 30초 이상 컵이 인식되지 않다가 다시 감지되면 동작
LastTimeDetectedCup = 0 # 가장 최근에 컵이 감지된 시각

def ActModeCup(repeat = 2):
    speed = 0.4
    delay = 0.05
    
    RGB.Set_BreathSColor_RGB(2) # Color : 0(red), 1(green), 2(blue), 3(yellow), 4(magenta), 5(cyan), 6(white)
    RGB.Set_BreathSSpeed_RGB(3) # speed : 1(slow), 2(normal), 3(fast)
    RGB.Set_BreathSLight_RGB()
    
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틸트 위치 center-up
    
    for i in range(0, repeat):
        # 물 마시는 동작 연출
        pantilt.Servo_serial_control(2, DEF_TILT-100) # down
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-200) # down
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-300) # down
        Move2(-speed, speed)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-400) # down
        Stop()
        time.sleep(delay)
        
        for j in range(0 , 3):
            pantilt.Servo_serial_control(2, DEF_TILT-600)
            time.sleep(delay*2)
            pantilt.Servo_serial_control(2, DEF_TILT-500)
            time.sleep(delay*2)
            
        # 물을 마신 다음 고개를 위로 들고 트림하는 동작 연출
        pantilt.Servo_serial_control(2, DEF_TILT-300)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-200)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT-100)
        time.sleep(delay)
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(delay)
        
        pantilt.Servo_serial_control(2, DEF_TILT+250)
        time.sleep(delay*2)
        pantilt.Servo_serial_control(2, DEF_TILT+500)
        time.sleep(delay*4)
        pantilt.Servo_serial_control(2, DEF_TILT+400)
        time.sleep(delay*2)
        pantilt.Servo_serial_control(2, DEF_TILT+500)
        time.sleep(delay*4)
        pantilt.Servo_serial_control(2, DEF_TILT+250)
        time.sleep(delay*2)
        
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(delay)
        
    # 고개를 좌우로 한 번 털어줌
    pantilt.Servo_serial_control(1, MID_PAN+200) # Left
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+400)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+200)
    time.sleep(delay)
    
    pantilt.Servo_serial_control(1, MID_PAN-200) # Right
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-400)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-200)
    time.sleep(delay)
    
    pantilt.Servo_serial_control(1, MID_PAN)
    
    RGB.OFF_ALL_RGB() # 전체 LED 끄기
    RGB.Set_All_RGB(0x00, 0x00, 0x10) # 기본색상 어두운 파랑

In [23]:
ActModeCup()

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
217
b'\xff\xff\x02\x07\x03*\x0e\xd8\x00\n\xd9'
61
b'\xff\xff\x02\x07\x03*\x0et\x00\n='
161
b'\xff\xff\x02\x07\x03*\x0e\x10\x00\n\xa1'
106
b'\xff\xff\x02\x07\x03*\rH\x00\nj'
6
b'\xff\xff\x02\x07\x03*\r\xac\x00\n\x06'
106
b'\xff\xff\x02\x07\x03*\rH\x00\nj'
6
b'\xff\xff\x02\x07\x03*\r\xac\x00\n\x06'
106
b'\xff\xff\x02\x07\x03*\rH\x00\nj'
6
b'\xff\xff\x02\x07\x03*\r\xac\x00\n\x06'
61
b'\xff\xff\x02\x07\x03*\x0et\x00\n='
217
b'\xff\xff\x02\x07\x03*\x0e\xd8\x00\n\xd9'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
16
b'\xff\xff\x02\x07\x03*\x0f\xa0\x00\n\x10'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
16
b'\xff\xff\x02\x07\x03*\x0f\xa0\x00\n\x10'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
217
b'\xff\xff

#### MODE_BOOK 동작: 책을 보면 공부하기 싫어서 놀라며 고개를 떨구고 가로 저음
책이 마지막에 감지된 시각으로부터 특정 시간보다 더 지나서 책이 감지되면 동작됨

In [21]:
ActTimePeriodBook = 5 # 30초 이상 책이 감지되지 않다가 다시 감지되면 동작
LastTimeDetectedBook = 0 # 가장 최근에 책이 감지된 시각

def ActModeBook(repeat=2):
    speed = 0.4
    delay = 0.05
    
    RGB.Set_BreathSColor_RGB(0) # Color : 0(red), 1(green), 2(blue), 3(yellow), 4(magenta), 5(cyan), 6(white)
    RGB.Set_BreathSSpeed_RGB(3) # speed : 1(slow), 2(normal), 3(fast)
    RGB.Set_BreathSLight_RGB()
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틸트 위치 center-up
        
    # 깜짝 놀라서 고개를 쳐들고 뒷걸음질 치며 움찔
    pantilt.Servo_serial_control(2, DEF_TILT+600) # up
    Beep(0.01)
    time.sleep(delay)
    pantilt.Servo_serial_control(2, DEF_TILT)
    time.sleep(delay)
    Beep(0.01)
    
    pantilt.Servo_serial_control(2, DEF_TILT+300) # up
    Move2(-speed, speed)
    time.sleep(delay*2)
    pantilt.Servo_serial_control(2, DEF_TILT+600)
    Move2(-speed, speed)
    time.sleep(delay*2)
    pantilt.Servo_serial_control(2, DEF_TILT+300)
    Move2(speed, speed)
    time.sleep(delay*2)
    pantilt.Servo_serial_control(2, DEF_TILT)
    Move2(speed, speed)
    time.sleep(delay*2)
    Stop()
    
    # 고개 좌우로 흔듦 (시러시러)
    for i in range(0, repeat):
        pantilt.Servo_serial_control(1, MID_PAN+300) # Left
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-300)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+300)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-300)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+200)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-200)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN+100)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN-100)
        time.sleep(delay)
        pantilt.Servo_serial_control(1, MID_PAN)
        time.sleep(delay)
        
    # 고개를 좌우로 떨구며 한숨 푹푹
    pantilt.Servo_serial_double_control(1, MID_PAN-200, 2, DEF_TILT-300)
    time.sleep(delay*2)
    pantilt.Servo_serial_double_control(1, MID_PAN-400, 2, DEF_TILT-600)
    Beep(0.01)
    time.sleep(delay*16)
    pantilt.Servo_serial_double_control(1, MID_PAN-200, 2, DEF_TILT-400)
    time.sleep(delay*4)
    pantilt.Servo_serial_double_control(1, MID_PAN-100, 2, DEF_TILT-200)
    time.sleep(delay*2)
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT)
    time.sleep(delay*8)
    
    pantilt.Servo_serial_double_control(1, MID_PAN+200, 2, DEF_TILT-300)
    time.sleep(delay*2)
    pantilt.Servo_serial_double_control(1, MID_PAN+400, 2, DEF_TILT-600)
    Beep(0.01)
    time.sleep(delay*16)
    pantilt.Servo_serial_double_control(1, MID_PAN+200, 2, DEF_TILT-400)
    time.sleep(delay*4)
    pantilt.Servo_serial_double_control(1, MID_PAN+100, 2, DEF_TILT-200)
    time.sleep(delay*2)
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT)
    time.sleep(delay*8)
    
    # 서서히 고개를 좌우로 흔들면서 싫다고 표현
    pantilt.Servo_serial_control(1, MID_PAN+50)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+100)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+150)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+200)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+250)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+300)
    Beep(0.05)
    time.sleep(delay*2)
    pantilt.Servo_serial_control(1, MID_PAN+250)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+200)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+150)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+50)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN)
    time.sleep(delay)
    
    pantilt.Servo_serial_control(1, MID_PAN-50)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-100)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-150)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-200)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-250)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-300)
    Beep(0.05)
    time.sleep(delay*2)
    pantilt.Servo_serial_control(1, MID_PAN-250)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-200)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-150)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-50)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN)
    time.sleep(delay)
    
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틸트 위치
    Beep(0.1)
    time.sleep(0.2)
    Beep(0.5)
    
    RGB.OFF_ALL_RGB() # 전체 LED 끄기
    RGB.Set_All_RGB(0x00, 0x00, 0x10) # 기본 색상 어두운 파랑
    

In [25]:
ActModeBook()

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
16
b'\xff\xff\x02\x07\x03*\x0f\xa0\x00\n\x10'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
177
b'\xff\xff\x02\x07\x03*\x0f\xff\x00\n\xb1'
16
b'\xff\xff\x02\x07\x03*\x0f\xa0\x00\n\x10'
40
b'\xff\xff\x01\x07\x03*\t\x8f\x00\n('
130
b'\xff\xff\x01\x07\x03*\x077\x00\n\x82'
40
b'\xff\xff\x01\x07\x03*\t\x8f\x00\n('
130
b'\xff\xff\x01\x07\x03*\x077\x00\n\x82'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
241
b'\xff\xff\x01\x07\x03*\x08\xc7\x00\n\xf1'
186
b'\xff\xff\x01\x07\x03*\x07\xff\x00\n\xba'
85
b'\xff\xff\x01\x07\x03*\x08c\x00\nU'
40
b'\xff\xff\x01\x07\x03*\t\x8f\x00\n('
130
b'\xff\xff\x01\x07\x03*\x077\x00\n\x82'
40
b'\xff\xff\x01\x07\x03*\t\x8f\x00\n('
130
b'\xff\xff\x01\x07\x03*\x077\x00\n\x82'
140
b'\xff\xff\x01\x07\x03*\t+\x00\n\x8c'
30
b'\xff\xff\x01\x07\x03*\x07\x9b\x00\n\x1e'
2

#### MODE_NONE 동작: 특정 시간동안 관심 물체가 하나도 감지되지 않을 때 심심해 하는 동작
대상 객체들이 하나도 감지되지 않는 시간이 특정 시간보다 길어지면 동작

In [22]:
ActTimePeriodNone = 5 # 30초 이상 아무것도 감지되지 않으면 동작
StartTimeDetectedNone = 0 # 대상 물체들이 감지되지 않기 시작한 시각

def ActModeNone(repeat=2):
    speed = 0.4
    delay = 0.05
    
    RGB.Set_WaterfallLight_RGB()
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틸트 위치
    
    # 꾸벅꾸벅 조는 행동 연출
    for i in range(0, repeat):
        for j in range(0, 200, 20):
            pantilt.Servo_serial_control(2, DEF_TILT-j) # down
            time.sleep(0.08)
        time.sleep(0.5)
        for j in range(200, 0, -20):
            pantilt.Servo_serial_control(2, DEF_TILT-j) # up
            time.sleep(0.03)
        time.sleep(2)
        
        for j in range(0, 300, 20):
            pantilt.Servo_serial_control(2, DEF_TILT-j) # down
            time.sleep(0.05)
        time.sleep(0.5)
        for j in range(300, 0, -20):
            pantilt.Servo_serial_control(2, DEF_TILT-j) # up
            time.sleep(0.02)
        time.sleep(2)
        
        for j in range(0, 300, 20):
            pantilt.Servo_serial_control(2, DEF_TILT-j) # down
            time.sleep(0.03)
        for j in range(300, 450, 20):
            pantilt.Servo_serial_control(2, DEF_TILT-j) # more down
            time.sleep(0.005)
            
        # 자다가 깜짝 놀라서 고개 치켜듦
        pantilt.Servo_serial_control(2, DEF_TILT-400)
        pantilt.Servo_serial_control(2, DEF_TILT-300)
        pantilt.Servo_serial_control(2, DEF_TILT-200)
        time.sleep(0.05)
        pantilt.Servo_serial_control(2, DEF_TILT-100)
        time.sleep(0.1)
        pantilt.Servo_serial_control(2, DEF_TILT)
        time.sleep(0.5)
        
    time.sleep(1)
    
    pantilt.Servo_serial_control(1, MID_PAN+200) # Left
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+400)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN+200)
    time.sleep(delay)
    
    pantilt.Servo_serial_control(1, MID_PAN-200) # Right
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-400)
    time.sleep(delay)
    pantilt.Servo_serial_control(1, MID_PAN-200)
    time.sleep(delay)
    
    pantilt.Servo_serial_control(1, MID_PAN)
    pantilt.Servo_serial_double_control(1, MID_PAN, 2, DEF_TILT) # 중앙상향 얼굴인식 팬틽트 위치
    
    RGB.OFF_ALL_RGB() # 전체 LED 끄기
    RGB.Set_All_RGB(0x00, 0x00, 0x10) # 기본 색상으로

In [27]:
ActModeNone()

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
16
b'\xff\xff\x02\x07\x03*\x0f\xa0\x00\n\x10'
36
b'\xff\xff\x02\x07\x03*\x0f\x8c\x00\n$'
56
b'\xff\xff\x02\x07\x03*\x0fx\x00\n8'
76
b'\xff\xff\x02\x07\x03*\x0fd\x00\nL'
96
b'\xff\xff\x02\x07\x03*\x0fP\x00\n`'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
136
b'\xff\xff\x02\x07\x03*\x0f(\x00\n\x88'
156
b'\xff\xff\x02\x07\x03*\x0f\x14\x00\n\x9c'
176
b'\xff\xff\x02\x07\x03*\x0f\x00\x00\n\xb0'
197
b'\xff\xff\x02\x07\x03*\x0e\xec\x00\n\xc5'
217
b'\xff\xff\x02\x07\x03*\x0e\xd8\x00\n\xd9'
197
b'\xff\xff\x02\x07\x03*\x0e\xec\x00\n\xc5'
176
b'\xff\xff\x02\x07\x03*\x0f\x00\x00\n\xb0'
156
b'\xff\xff\x02\x07\x03*\x0f\x14\x00\n\x9c'
136
b'\xff\xff\x02\x07\x03*\x0f(\x00\n\x88'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
96
b'\xff\xff\x02\x07\x03*\x0fP\x00\n`'
76
b'\xff\xff\x02\x07\x03*\x0fd\x00\nL'
56
b'\xff\xff\x02\x07\x03*\x0fx\x00\n8'
36
b'\xff\xff\x02\x07\x03*\x0f\x8c\x00\n$'
16
b'\xff\xff\x02\x07\x03*\x0f\xa0\x00\n\x10'
36
b'\xff\xff\

### Object Follow 알고리즘

#### 단일 카메라 이미지 검출

In [23]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

Beep(0.1)
time.sleep(0.25)
Beep(0.1)

In [24]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

#### 검출된 내용을 텍스트 영역에 프린트하기
아래 코드는 검출된 내용을 프린트.
영상 프레임 이미지에서 물체가 검출될 때까지 기다림(아래 내용 프린트 시 오류발생 방지)

In [25]:
import time
detections = model(camera.value)

while str(detections) == '[[]]':
    print('no detection')
    time.sleep(1)
    detections = model(camera.value)
    
print(detections)

[[{'label': 1, 'confidence': 0.9015745520591736, 'bbox': [0.1257682740688324, 0.289200097322464, 0.8719518184661865, 0.979526162147522]}]]


##### 영상 이미지에서 검출된 물체의 label, confidence, 그리고 경계서 위치를 텍스트 위젯에 보여줌

In [26]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

Textarea(value="[[{'label': 1, 'confidence': 0.9015745520591736, 'bbox': [0.1257682740688324, 0.28920009732246…

##### 첫번째 이미지에서 검출된 첫 물체의 검출 내용을 확인하는 코드
검출된 물체 없을 경우에는 오류메시지 출력

In [27]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

{'label': 1, 'confidence': 0.9015745520591736, 'bbox': [0.1257682740688324, 0.289200097322464, 0.8719518184661865, 0.979526162147522]}


### 잿봇이 대상 물체 따라다니게 만들기
절차
1. 지정 물체 클래스를 검출
2. 카메라 뷰의 중심에 가장 가까이에서 검출된 물체를 대상으로 함
3. 중심점에서 떨어진 거리만큼 젯봇 이동
4. 만약 젯봇이 장애물 만나면 살짝 좌회전

##### 제어를 위해 대상 물체 레이블이나 주행 속력과 회전 게인 생성

In [76]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    #Image zoomed to 224,224 versus 224,244 obstacle avoidance model
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

##### PID 제어 모듈을 사용하여 제어 인스턴스를 만들고 초기화

In [77]:
import PID

global follow_speed
follow_speed = Speed # 기본속력 0.7

global turn_gain
turn_gain = 1.5

global follow_speed_pid, follow_speed_pid_model
follow_speed_pid_model = 1

# follow_speed_pid = PID.PositionalPID(3, 0, 0)
follow_speed_pid = PID.PositionalPID(1.5, 0, 0.05)

global turn_gain_pid
turn_gain_pid = PID.PositionalPID(0.15, 0, 0.05)

global object_yservo_pid
object_yservo_pid = PID.PositionalPID(3, 0.5, 0)

##### 객체 검출 여부 및 미검출 지속 시간

In [78]:
import time

detectedPerson = 0 # index 1

detectedDog = 0 # 18
detectedStop = 0 # 13

detectedCup = 0 # 47
detectedFruit = 0 # 52, 53, 55
detectedBook = 0 # 84

MODE_NONE = 0
MODE_PERSON = 1
MODE_FOLLOW = 2
MODE_CUP = 3
MODE_FRUIT = 4
MODE_BOOK = 5

MIN_KEEP_MODE_TIME = 2.0
actionMode = MODE_NONE
modeTime = time.time()   # 현재 모드로 전환된 시간

# 첫 실행될 때 바로 첫번째 검출 객체에 대한 감성동작이 실행되는 것을 방지하기 위해
# 객체들의 마지막 검출시각을 모두 현재 시각으로 설정
LastTimeDetectedPerson = LastTimeDetectedFruit = LastTimeDetectedCup = modeTime
LastTimeDetectedBook = StartTimeDetectedNone = modeTime

# 동일한 감성동작이 연속해서 반복되지 않도록 마지막 동작 기억
LastPerformedAct = MODE_NONE

##### 정의된 모든 위젯을 디스플레이 하고, 카메라 영상을 연결하여 메인 검출 신경망 실행

In [79]:
from jetbot import bgr8_to_jpeg

if runMode < 3:
    image_widget = widgets.Image(format='jpeg', width=300, height=300)
    label_widget = widgets.IntText(value=1, description='target#') # person : 1
    detected_widget = widgets.Text(value='', description='detected#')

    display(widgets.VBox([
        widgets.HBox([image_widget]),
        label_widget, detected_widget
    ]))

##### 검출된 객체의 중심 좌표를 얻는 함수와 2차원 벡터의 길이를 구하는 함수, 영상 화면 중심에서 가장 가까이서 검출된 객체 찾는 함수 선언

In [80]:
def detection_center(detection):
    """Calculate the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Calculate the length of a two-dimensional vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Find the detection closest to the center of the image"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

##### 프로그램의 메인 루프 코드. 영상 이미지로부터 계속 반복해서 객체를 검출하고, 검출된 객체와 시간에 따라 해당 감성동작을 호출하고, 
PID 제어로 카메라 팬/틸트 각도를 제어하여 추적 대상 물체를 추적하며, 강아지가 검출된 모드에서는 사람을 따라 주행

In [81]:
def execute(change):
    
    global follow_speed
    global turn_gain
    global follow_speed_pid
    
    target_value_speed = 0
    
    #Update image value
    image = change['new']
    
    global ActTimePeriodPerson
    global LastTimeDetectedPerson
    global ActTimePeriodFruit
    global LastTimeDetectedFruit
    global ActTimePeriodCup
    global LastTimeDetectedCup
    global ActTimePeriodBook
    global LastTimeDetectedBook
    global ActTimePeriodFruit

    global ActTimePeriodNone
    global StartTimeDetectedNone
    global LastPerformedAct
    
    global runMode
    global actionMode
    global modeTime
    
    global detectedPerson
    global detectedDog
    global detectedStop
    global detectedCup
    global detectedFruit
    global detectedBook
    
    # 검출된 내용물 받음
    detections = model(image)
    
    # 물체 확인
    for det in detections[0]:
        bbox = det['bbox']
        
        bDrawBox = 1 # 일단 검출 박스를 그림
        
        if det['label'] == 1:
            detectedPerson += 1
        elif det['label'] == 18:
            detectedDog += 1
        elif det['label'] == 13:
            detectedStop += 1
        elif det['label'] == 47:
            detectedCup += 1
        elif (det['label'] == 52) or (det['label'] == 53) or (det['label'] == 55):
            detectedFruit += 1
        elif det['label'] == 84:
            detectedBook += 1
        else:
            bDrawBox = 0 # 원하는 객체 아니면 box 안그림
            
        if runMode < 2:
            if bDrawBox :
                cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), 
                              (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
                
        if runMode < 2:
            detects = ''
            if detectedPerson:
                detects += 'Person '
            if detectedDog:
                detects += 'Dog '
            if detectedStop:
                detects += 'Stop '
            if detectedCup:
                detects += 'Cup '
            if detectedBook:
                detects += 'Book'
            detected_widget.value = detects
            
            # 어떤 물체가 검출되었을 때, 이전의 물체 검출시각으로 부터 Min 시간이 지나야 전환 가능
            # 모드가 바뀌면, 마지막모드 전환 시각으로부터 특정 시간이 경과하면 해당 동작 1회 실행
            
        newTime = time.time() # 시간 비교 위해
            
        if detectedCup:
            if (actionMode != MODE_FOLLOW) and (actionMode != MODE_CUP) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                if newTime - LastTimeDetectedCup > ActTimePeriodCup:
                    if LastPerformedAct != MODE_CUP:
                        ActModeCup()
                        LastPerformedAct = MODE_CUP
                modeTime = LastTimeDetectedCup = newTime
                actionMode = MODE_CUP
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("CUP : " + str(detectedCup))
                        
        elif detectedFruit:
            if (actionMode != MODE_FOLLOW) and (actionMode != MODE_FRUIT) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                if newTime - LastTimeDetectedFruit > ActTimePeriodFruit:
                    if LastPerformedAct != MODE_FRUIT:
                        ActModeCup()
                        LastPerformedAct = MODE_FRUIT
                modeTime = LastTimeDetectedFruit = newTime
                actionMode = MODE_FRUIT
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("FRUIT : " + str(detectedFruit))

        elif detectedBook:
            if (actionMode != MODE_FOLLOW) and (actionMode != MODE_BOOK) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                if newTime - LastTimeDetectedBook > ActTimePeriodBook:
                    if LastPerformedAct != MODE_BOOK:
                        ActModeCup()
                        LastPerformedAct = MODE_BOOK
                modeTime = LastTimeDetectedBook = newTime
                actionMode = MODE_BOOK
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("BOOK : " + str(detectedBook))
        
        elif detectedDog:
            if (actionMode != MODE_FOLLOW) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                modeTime = newTime
                actionMode = MODE_FOLLOW
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("FOLLOW" + str(detectedDog))
                    
        elif detectedStop:
            if (actionMode != MODE_PERSON) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                modeTime = LastTimeDetectedStop = newTime
                actionMode = MODE_PERSON
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("PERSON!" + str(detectedPerson))
                    
        elif detectedPerson:
            if (actionMode != MODE_FOLLOW) and (actionMode != MODE_PERSON) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                if newTime - LastTimeDetectedPerson > ActTimePeriodPerson:
                    if LastPerformedAct != MODE_PERSON:
                        ActModeCup()
                        LastPerformedAct = MODE_PERSON
                modeTime = LastTimeDetectedPerson = newTime
                actionMode = MODE_PERSON
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("PERSON : " + str(detectedPerson))
                    
        else:
            if actionMode != MODE_FOLLOW:
                
                if actionMode == MODE_NONE:
                    if newTime - LastTimeDetectedPerson > ActTimePeriodNone:
                        if LastPerformedAct != MODE_NONE:
                            ActModeCup()
                            LastPerformedAct = MODE_NONE
                        StartTimeDetectedNone = newTime
                
                else:
                    if newTime - modeTime > MIN_KEEP_MODE_TIME:
                        modeTime = StartTimeDetectedNone = newTime
                        actionMode = MODE_NONE
                        if runMode < 2:
                            ShowTopInfo("(none)")
        
        matching_detections = [d for d in detections[0] if d['label'] == 1]
        
        det = closest_detection(matching_detections)
        
        if det is None:
            Stop() # 검출 안되면 정지
            
        else:
            if runMode < 2:
                bbox = det['bbox']
                cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), 
                              (int(300 * bbox[2]), int(300 * bbox[3])), (0, 255, 0), 4)
            
            if actionMode == MODE_FOLLOW:
                
                # 중심에서 대산 물체의 x축 오프셋 만큼 이동
                center = detection_center(det)
                
                # 따라가는 속도 제어
                follow_speed_pid.SystemOutput = 76000 * (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
                
                follow_speed_pid.SetStepSignal(30000)
                follow_speed_pid.SetInertiaTime(0.2, 0.1)
                
                #Steering gain PID adjustment
                turn_gain_pid.SystemOutput = center[0]
                turn_gain_pid.SetStepSignal(0)
                turn_gain_pid.SetInertiaTime(0.2, 0.1)
                
                target_value_turn_gain = 0.5 + abs(turn_gain_pid.SystemOutput)
                
                if target_value_turn_gain < 0:
                    target_value_turn_gain = 0
                elif target_value_turn_gain > 2:
                    target_value_turn_gain = 2
                    
                # 바퀴 모터 속력
                target_value_speed = 0.5 + follow_speed_pid.SystemOutput / 76000
                
                target_value_speedl = target_value_speed + target_value_turn_gain * center[0]
                target_value_speedr = target_value_speed - target_value_turn_gain * center[0]
                
                if target_value_speedl < 0.4:
                    target_value_speedl = 0
                    
                elif target_value_speedl > 1:
                    target_value_speedl = 1
                    
                if target_value_speedr < 0.4:
                    target_value_speedr = 0
                    
                elif target_value_speedr > 1:
                    target_value_speedr = 1
                
                Move2(target_value_speedl, target_value_speedr)
            
        if runMode < 3:
            # 이미지 디스플레이 업데이트
            image_widget.value = bgr8_to_jpeg(image)
            
### execute 실험!!

modeTime = time.time()
actionMode = MODE_NONE

execute({'new': camera.value})

17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
11617
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'

b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
116
b'\xff\xff\x02\x07\x03*\x0f<\x00\nt'
217
b'\xff\xff\x02\x07\x03*\x0e\xd8\x00\n\xd9'
217
b'\xff\xff\x02\x07\x03*\x0e\xd8\x00\n\xd9'
217
b'\xff\xff\x02\x07\x03*\x0e\xd8\x00\n\xd9'
61
b'\xff\xff\x02\x07\x03*\x0et\x00\n='
61
b'\xff\xff\x02\x07\x03*\x0et\x00\n='
61
b'\xff\xff\x02\x07\x03*\x0et\x00\n='
161
b'\xff\xff\x02\x07\x03*\x0e\x10\x00\n\xa1'
161
b'\xff\xff\x02\x07\x03*\x0e\x10\x00\n\xa1'
161
b'\xff\xff\x02\x07\x03*\x0e\x10\x00\n\xa1'
106
b'\xff\xff\x02\x07\x03*\rH\x00\nj'
106
b'\xff\xff\x02\x07\x03*\rH\x00\nj'
106
b'\xff\xff\x02\x07\x03*\rH\x00\nj'
6
b'\xff\xff\x02\x07\x03*\r\xac\x00\n\x06'
6
b'\xff\xff\x02\x07\x03*\r\xac\x00\n\x06'
6
b'\xff\xff\x02\x07\x03*\r\xac\x00\n\x06'
106
b'\xff\xff\x02\x07

In [90]:
import threading
import inspect
import ctypes

''' definition of the close thread function'''

def _async_raise(tid, exctype):
    
    '''raises the exception, performs cleanup if needed'''
    tid = ctypes.c_long(tid)
    
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    
    if res == 0:
        raise ValueError('invalid thread id')
    
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble 
        # and you should call it again with exc=NULL to revert the effect """
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)
    
# thread를 만들어 객체 검출 함수를 반복해서 호출
def test():
    while True:
        execute({'new': camera.value})
        
thread1 = threading.Thread(target=test)
thread1.setDaemon(False)
thread1.start()

In [89]:
Beep(0.1)

RuntimeError: Please set pin numbering mode using GPIO.setmode(GPIO.BOARD), GPIO.setmode(GPIO.BCM), GPIO.setmode(GPIO.TEGRA_SOC) or GPIO.setmode(GPIO.CVM)

In [91]:
stop_thread(thread1)
camera.unobserve_all()
robot.stop()
Stop()


In [92]:
robot.stop()
Stop()

In [86]:
GPIO.cleanup()
Beep(0.5)

RuntimeError: Please set pin numbering mode using GPIO.setmode(GPIO.BOARD), GPIO.setmode(GPIO.BCM), GPIO.setmode(GPIO.TEGRA_SOC) or GPIO.setmode(GPIO.CVM)